In [2]:
import sklearn as sk
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

Load dataset

In [3]:
digits = load_digits()
X,y = digits.data, digits.target

In [4]:
X.shape, y.shape

((1797, 64), (1797,))

Split dataset into 70% and 30%

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

Split 30% into 15% calidation and 15% test

In [6]:
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

Standarize features

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

In [10]:
class GaussianGenerativeModel():
    def __init__(self,x_train,y_train):
        self.x_train = x_train
        self.y_train = y_train
        self.d = self.x_train.shape[1]
        self.pi = [] # Class priors
        self.mu = [] # Class means
        self.sigma = np.zeros((d, d)) # Covariance matrix
        self.sigma_reg = np.zeros((d, d)) # Regularized covariance matrix
    def _estimate_class_priors(self):
        n = len(self.y_train)
        for k in range(10):
            pi_k = self.y_train.count(k)/n
            self.pi.append(pi_k)
        return self.pi
    def _estimate_class_means(self):
        for k in range(10):
            x_class = self.x_train[self.y_train == k] # Get all samples that are equal to k
            mu_k = x_class.mean(axis=0) # return a vector of size 64x1
            self.mu.append(mu_k)
    def _estimate_covariance_matrix(self):
        self.sigma = np.zeros((d, d)) 
        # Iterate over training samples
        for i in range(len(self.y_train)):
            # compute the difference
            diff = self.x_train[i] - self.mu[self.y_train[i]]
            # create a diagonal matrix
            self.sigma += np.outer(diff,diff)
        self.sigma /= len(self.y_train)
    def regularize(self,reg_term=0):
        self.sigma_reg = self.sigma + reg_term * np.eye(d)
    def fit(self):
        self._estimate_class_priors()
        self._estimate_class_means()
        self._estimate_covariance_matrix()
        
    def predict(self):
        scores = np.log(pi)